In [1]:
#Luiz Felipe Barros Alves
#12/06/2025

In [2]:
import sys, os, imp
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import lambda2nu, nu2lambda, c

C:\Users\alves\AppData\Local\Temp\ipykernel_16260\3778166921.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import sys, os, imp


In [9]:
# lumapiFile = "/opt/lumerical/v221/api/python/lumapi.py"
lumapiFile = "C:\\Program Files\\Lumerical\\v242\\api\\python\\lumapi.py"
lumapi = imp.load_source("lumapi", lumapiFile)
fdtdApi = lumapi.FDTD(hide = False)

In [4]:
# units
um = 1e-6
nm = 1e-9

sub_material = 'SiO2 (Glass) - Palik'
core_material = 'Si (Silicon) - Palik'

gap = 200*nm

y_span_core =  450*nm
z_span_core =  220*nm

x_core = 0*um
y_core = y_span_core+gap
z_core = 0.0*um

wg90Raius = 10*um
y_span_guia_reto = 2*um

## Banda de operação
wvlngth_start = 1500 * nm
wvlngth_stop = 1600 * nm
nmbFrqPnts = 21

## Parâmetros do solver FDTD
# Tempo de simulação
simulation_time = (np.pi * wg90Raius + 47.5*um)*7/ c + 200e-15

# Dimensões do volume do solver
x_FDTD = 0
y_FDTD = 0

y_span_FDTD = wg90Raius + 2*um + 1.5*um
z_span_FDTD = 4*um

# Material
backGroundMaterial = sub_material

# Precisão do mesh
mesh_accuracy = 2

# Condições de fronteira
z_min_bc = "Symmetric"


# Parametros das portas

y_span_port = y_span_core*4
z_span_port = z_span_core*4
# mode_selection = "fundamental TE mode"
injection_axis_1 = "y"
injection_axis_2 = "x"

# Create Directional Coupler

In [5]:
def create_dc(x_span_core:float): 
  
    fdtdApi.switchtolayout()
    fdtdApi.deleteall()

    # Adicionando guias retos - acoplador
    fdtdApi.addrect()
    fdtdApi.set("name", "Lc")
    fdtdApi.set("material", core_material)
    fdtdApi.set("x", x_core)
    fdtdApi.set("x span", x_span_core)
    fdtdApi.set("y", y_core)
    fdtdApi.set("y span", y_span_core)
    fdtdApi.set("z", z_core)
    fdtdApi.set("z span", z_span_core)

    fdtdApi.copy()
    fdtdApi.set("name", "Lc_lower")
    fdtdApi.set("y", 0)
    fdtdApi.set('x span', x_span_core + wg90Raius*2 + 4*um)

    # Adicionando guia bend
    fdtdApi.addobject("90_bend_wg")
    fdtdApi.set("name", "right")
    fdtdApi.set("base height", z_span_core)
    fdtdApi.set("base width", y_span_core)
    fdtdApi.set("first axis", 'z')
    fdtdApi.set('radius', wg90Raius)
    fdtdApi.set('rotation 1', -90)
    fdtdApi.set('x', x_span_core/2)
    fdtdApi.set('y', y_core+wg90Raius)
    fdtdApi.set('z', z_core)
    fdtdApi.set('material', core_material)

    fdtdApi.copy()
    fdtdApi.set("name", "left")
    fdtdApi.set('rotation 1', 180)
    fdtdApi.set('x', -x_span_core/2)

    # Adicionando guias retos auxiliares
    fdtdApi.addrect()
    fdtdApi.set("name", "wg_1")
    fdtdApi.set("material", core_material)
    fdtdApi.set("x", -(x_span_core + wg90Raius*2)/2)
    fdtdApi.set("x span", y_span_core)
    fdtdApi.set("y", wg90Raius+y_core+y_span_guia_reto/2)
    fdtdApi.set("y span", y_span_guia_reto)
    fdtdApi.set("z", z_core)
    fdtdApi.set("z span", z_span_core)

    fdtdApi.copy()
    fdtdApi.set("name", "wg_2")
    fdtdApi.set("x", (x_span_core + wg90Raius*2)/2)


# Create Sovers and monitors


In [6]:
def create_fdtd(x_span_FDTD:float, simulation_time:float):   
    
    fdtdApi.switchtolayout()
    fdtdApi.select('FDTD')
    fdtdApi.delete()
    fdtdApi.select('monitor')
    fdtdApi.delete()

    # Adiciona o solver
    fdtdApi.addfdtd()
    fdtdApi.set("x", x_FDTD)
    fdtdApi.set("x span", x_span_FDTD)
    fdtdApi.set("y", wg90Raius/2)
    fdtdApi.set("y span", y_span_FDTD)
    fdtdApi.set("z", 0)
    fdtdApi.set("z span", z_span_FDTD)
    fdtdApi.set("background material", backGroundMaterial)
    fdtdApi.set("mesh accuracy", mesh_accuracy)
    fdtdApi.set("simulation time", simulation_time)
    fdtdApi.set("z min bc", z_min_bc)

    fdtdApi.addprofile();
    fdtdApi.set('name', 'monitor');
    fdtdApi.set("x", x_FDTD)
    fdtdApi.set("x span", x_span_FDTD)
    fdtdApi.set("y", wg90Raius/2)
    fdtdApi.set("y span", y_span_FDTD)
    fdtdApi.set("z", 0)
    fdtdApi.set("override global monitor settings", True)
    fdtdApi.set("frequency points", nmbFrqPnts)

    # fdtdApi.addmovie();
    # fdtdApi.set('name', 'movie');
    # fdtdApi.set("x", x_FDTD)
    # fdtdApi.set("x span", x_span_FDTD)
    # fdtdApi.set("y", wg90Raius/1.7)
    # fdtdApi.set("y span", y_span_FDTD)
    # fdtdApi.set("z", 0)

# Add ports

In [7]:
def add_ports(x_span_core:float): 
   # Adiciona portas na simulação
    fdtdApi.switchtolayout()
    fdtdApi.select('FDTD::ports')
    fdtdApi.delete()


    fdtdApi.addport()
    fdtdApi.set("name", "port upper 1")
    fdtdApi.set("injection axis", injection_axis_1)
    fdtdApi.set("x", -(x_span_core + wg90Raius*2)/2)
    fdtdApi.set("y", wg90Raius + 1.2*um)
    fdtdApi.set("z", 0)
    fdtdApi.set("x span", y_span_port)
    fdtdApi.set("z span", z_span_port)
    # fdtdApi.set("mode selection", mode_selection)
    fdtdApi.set('direction', 'Backward')

    fdtdApi.copy()
    fdtdApi.set("name", "port upper 2")
    fdtdApi.set("x", (x_span_core + wg90Raius*2)/2)

    fdtdApi.addport()
    fdtdApi.set("name", "port lower 2")
    fdtdApi.set("injection axis", injection_axis_2)
    fdtdApi.set("x", (x_span_core + wg90Raius*2)/2)
    fdtdApi.set("y", 0)
    fdtdApi.set("z", 0)
    fdtdApi.set("y span", y_span_port)
    fdtdApi.set("z span", z_span_port)
    # fdtdApi.set("mode selection", mode_selection)
    fdtdApi.set('direction', 'Backward')

    fdtdApi.copy()
    fdtdApi.set("name", "port lower 1")
    fdtdApi.set("x", -(x_span_core + wg90Raius*2)/2)
    fdtdApi.set('direction', 'Forward')

# Configuration of Simulation

In [ ]:
x_span_array = np.linspace(40, 47.5, 20)*um

for x_span_core, i in zip(x_span_array, range(17, 21)):

    simulation_time = (np.pi * wg90Raius + x_span_core)*7/ c + 200e-15
    x_span_FDTD = x_span_core + wg90Raius*2 + 2*um

    create_dc(x_span_core=x_span_core)
    create_fdtd(x_span_FDTD=x_span_FDTD, simulation_time=simulation_time)
    add_ports(x_span_core=x_span_core)

    # Configura a banda de simulação
    fdtdApi.setnamed("FDTD", "global source wavelength start", wvlngth_start)
    fdtdApi.setnamed("FDTD", "global source wavelength stop", wvlngth_stop)
    fdtdApi.setnamed("FDTD::ports", "monitor frequency points", nmbFrqPnts)

    fdtdApi.save('Simulacao_FDTD')
    fdtdApi.run()

    sweep_name = f'Simulation_{i}'

    fdtdApi.addsweep(3)
    fdtdApi.setsweep("s-parameter sweep", "name", sweep_name)
    fdtdApi.setsweep(sweep_name, "Excite all ports", 1)

    modestruct = {"label": "mode 1", "id" : 1}
    rowstruct = {"mode 1": modestruct, "location": "RIGHT"}

    portstruct = {"port lower 2": rowstruct}
    fdtdApi.setsweep(sweep_name, "export setup", portstruct)
    portstruct = {"port upper 2": rowstruct}
    fdtdApi.setsweep(sweep_name, "export setup", portstruct)

    fdtdApi.runsweep(sweep_name)

    txt_name = f'dc_gap=200nm_Lc={x_span_core/um}um.txt'
    dat_name = f'dc_gap=200nm_Lc={x_span_core/um}um.dat'

    fdtdApi.exportsweep(sweep_name, txt_name)
    fdtdApi.exportsweep(sweep_name, dat_name)